In [12]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import lightgbm as lgb
import seaborn as sns
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression


train_path = '/Users/yuichikuriyama/Desktop/AI Quest/train (3).csv'
test_path = '/Users/yuichikuriyama/Desktop/AI Quest/test (3).csv'

df = pd.read_csv(train_path)
df_test = pd.read_csv(test_path)

In [13]:
df["room_type"] = df["room_type"].replace("Shared room",1.0).replace("Private room",2.0).replace("Entire home/apt",3.0)
df["room_type"] = df["room_type"].astype(float).astype(int)

df_test["room_type"] = df_test["room_type"].replace("Shared room",1.0).replace("Private room",2.0).replace("Entire home/apt",3.0)
df_test["room_type"] = df_test["room_type"].astype(float).astype(int)



df["cancellation_policy"] = df["cancellation_policy"].replace("flexible","1.0").replace("moderate","2.0").replace("strict","3.0").replace("super_strict_30","4.0").replace("super_strict_60","5.0")
df_test["cancellation_policy"] = df_test["cancellation_policy"].replace("flexible","1.0").replace("moderate","2.0").replace("strict","3.0").replace("super_strict_30","4.0").replace("super_strict_60","5.0")

df["cancellation_policy"] = df["cancellation_policy"].astype(float).astype(int)
df_test["cancellation_policy"] = df_test["cancellation_policy"].astype(float).astype(int)


df["bed_type"] = df["bed_type"].replace("Couch",1.0).replace("Futon",2.0).replace("Airbed",3.0).replace("Pull-out Sofa",4.0).replace("Real Bed",5.0)
df_test["bed_type"] = df_test["bed_type"].replace("Couch",1.0).replace("Futon",2.0).replace("Airbed",3.0).replace("Pull-out Sofa",4.0).replace("Real Bed",5.0)

df["bed_type"] = df["bed_type"].astype(float).astype(int)
df_test["bed_type"] = df_test["bed_type"].astype(float).astype(int)

df['url'] = (df.thumbnail_url.isna()).astype('int')
df_test['url'] = (df.thumbnail_url.isna()).astype('int')

df["city"] = df["city"].replace("NYC","1.0").replace("LA","2.0").replace("SF","3.0").replace("DC","4.0").replace("Chicago","5.0").replace("Boston","6.0")
df_test["city"] = df_test["city"].replace("NYC","1.0").replace("LA","2.0").replace("SF","3.0").replace("DC","4.0").replace("Chicago","5.0").replace("Boston","6.0")


df["city"] = df["city"].astype(float).astype(int)
df_test["city"] = df_test["city"].astype(float).astype(int)




def change_tf(x):
    if  x=="t":
        return 1
    elif x=="f":
        return 0
df['cleaning_fee'] = df['cleaning_fee'].apply(change_tf)
df_test['cleaning_fee'] = df_test['cleaning_fee'].apply(change_tf)



df['url'] = (df.thumbnail_url.isna()).astype('int')
df_test['url'] = (df_test.thumbnail_url.isna()).astype('int')

df['first_review'] = (df.first_review.isna()).astype('int')
df_test['first_review'] = (df_test.first_review.isna()).astype('int')



df["cleaning_fee"] = df["cleaning_fee"].astype(float).astype(int)
df_test["cleaning_fee"] = df_test["cleaning_fee"].astype(float).astype(int)


In [14]:
df["multiple"] = df["number_of_reviews"]*df["review_scores_rating"]
df_test["multiple"] = df_test["number_of_reviews"]*df_test["review_scores_rating"]

In [24]:
X= df[["accommodates","beds","bathrooms","bedrooms","cancellation_policy","room_type","bed_type","city","multiple","cleaning_fee","first_review"]]
y=  df["y"]

In [16]:
from sklearn.model_selection import train_test_split, KFold, cross_validate, cross_val_score
kf = KFold(n_splits=4, shuffle=True, random_state=71)
for tr_idx,va_idx in kf.split(X,y):
    tr_x,va_x =X.iloc[tr_idx], X.iloc[va_idx]
    tr_y,va_y =y.iloc[tr_idx], y.iloc[va_idx]
    

In [17]:
model = lgb.LGBMRegressor()
model.fit(tr_x,tr_y)

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.1, max_depth=-1,
              min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
              n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
              random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
              subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [107]:
from sklearn.model_selection import GridSearchCV
model = lgb.LGBMRegressor()

param_grid = {"max_depth": [3,5,7,10, 25],
              "learning_rate" : [0.01,0.05,0.1],
              "num_leaves": [300,900],
              "n_estimators": [100,200,500,]
             }
# パラメータチューニングをグリッドサーチで行うために設定する
## このGridSearchCV には注意が必要 scoring は そのスコアを基準にして最適化する
grid_result = GridSearchCV(estimator = model,
                           param_grid = param_grid,
                           scoring =  "neg_mean_squared_error",
                           cv = 5,
                           verbose=3,
                           return_train_score = True,
                           n_jobs = -1)

grid_result.fit(tr_x,tr_y)

Fitting 5 folds for each of 90 candidates, totalling 450 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:   24.4s
[Parallel(n_jobs=-1)]: Done 120 tasks      | elapsed:  7.6min
[Parallel(n_jobs=-1)]: Done 280 tasks      | elapsed: 15.2min
[Parallel(n_jobs=-1)]: Done 450 out of 450 | elapsed: 21.0min finished


GridSearchCV(cv=5, error_score=nan,
             estimator=LGBMRegressor(boosting_type='gbdt', class_weight=None,
                                     colsample_bytree=1.0,
                                     importance_type='split', learning_rate=0.1,
                                     max_depth=-1, min_child_samples=20,
                                     min_child_weight=0.001, min_split_gain=0.0,
                                     n_estimators=100, n_jobs=-1, num_leaves=31,
                                     objective=None, random_state=None,
                                     reg_alpha=0.0, reg_lambda=0.0, silent=True,
                                     subsample=1.0, subsample_for_bin=200000,
                                     subsample_freq=0),
             iid='deprecated', n_jobs=-1,
             param_grid={'learning_rate': [0.01, 0.05, 0.1],
                         'max_depth': [3, 5, 7, 10, 25],
                         'n_estimators': [100, 200, 500],
      

In [18]:
y_pred_train = model.predict(tr_x)
from sklearn.metrics import mean_squared_error as MSE
mse = MSE(y_pred_train,tr_y )
rmse_train = np.sqrt(mse)
print(rmse_train)

105.77588102492159


In [19]:
y_pred_test = model.predict(va_x)

mse_test = MSE(va_y, y_pred_test)
rmse_test = np.sqrt(mse_test)
print(rmse_train)
print(rmse_test)


105.77588102492159
124.70020239110289


In [23]:
X_test= df_test[["accommodates","beds","bathrooms","bedrooms","cancellation_policy","room_type","bed_type","city","multiple","cleaning_fee","first_review"]]

In [21]:
test_pred = model.predict(X_test)

In [22]:
sample = pd.read_csv("/Users/yuichikuriyama/Desktop/AI Quest/sample_submit (1).csv",header=None)
sample[1] = test_pred
sample.to_csv("/Users/yuichikuriyama/Desktop/AI Quest/sample_submit (1).csv", index=None, header=None)